In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Physical_Exercise_Response/GSE156247'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Comparative transcriptome analysis of human skeletal muscle in response to cold acclimation and exercise training in human volunteers. [A294]"
!Series_summary	"Background: Cold acclimation and exercise training were previously shown to increase peripheral insulin sensitivity in human volunteers with type 2 diabetes. Although cold is a potent activator of brown adipose tissue, the increase in peripheral insulin sensitivity by cold is largely mediated by events occurring in skeletal muscle and at least partly involves GLUT4 translocation, as is also observed for exercise training. Results: To investigate if cold acclimation and exercise training overlap in the molecular adaptive response in skeletal muscle, we performed transcriptomics analysis on vastus lateralis muscle collected from human subjects before and after 10 days of cold acclimation, as well as before and after a 12-week exercise training intervention. Methods: Cold acclimation altered t

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

import pandas as pd

# Sample Characteristics Data (converted to DataFrame)
data = {
    0: ['subject id: GL23', 'subject id: RJ28', 'subject id: RP12', 'subject id: HE24', 'subject id: JW34', 'subject id: LD21', 
        'subject id: GS25', 'subject id: LB36', 'subject id: JS22', 'subject id: LC26', 'subject id: FL37', 'subject id: RT1', 
        'subject id: DG51', 'subject id: RN38', 'subject id: PP58', 'subject id: RS59', 'subject id: BE50', 'subject id: HM67'],
    1: ['disease: healthy'] * 18,
    2: ['time point: before training intervention period', 'time point: after training intervention period'] * 9,
    3: ['Sex: male'] * 18
}
clinical_data = pd.DataFrame(data)

# Check if the dataset contains gene expression data
is_gene_available = True  # Based on the information given, this is a microarray analysis of gene expression.

# Check availability of 'Physical_Exercise_Response', 'age', and 'gender' variables in the sample characteristics dictionary
trait_row = 2  # since 'time point' is available
age_row = None  # age data is not available
gender_row = 3  # gender data is available as all are male

# Define data type conversion functions
def convert_trait(value):
    value = value.split(':')[-1].strip()
    return 1 if value == "after training intervention period" else 0

def convert_age(value):
    return None  # Age data is not provided in the sample characteristics dictionary.

def convert_gender(value):
    value = value.split(':')[-1].strip()
    return 1 if value == "male" else None  # No females reported

# Save cohort information
save_cohort_info('GSE156247', './preprocessed/Physical_Exercise_Response/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Physical_Exercise_Response', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Physical_Exercise_Response/trait_data/GSE156247.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{0: [0, None], 1: [0, None], 2: [0, None], 3: [0, 1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['7892501', '7892502', '7892503', '7892504', '7892505', '7892506',
       '7892507', '7892508', '7892509', '7892510', '7892511', '7892512',
       '7892513', '7892514', '7892515', '7892516', '7892517', '7892518',
       '7892519', '7892520'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001005240,NM_001004195,NM_001005484,BC136848,BC136907', 'BC118988,AL137655', 'NM_001005277,NM_001005221,NM_001005224,NM_001005504,BC137547'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [53049.0, 63015.0, 69091.0, 334129.0, 367659.0], 'RANGE_STOP': [54936.0, 63887.0, 70008.0, 334296.0, 368597.0], 'total_probes': [7.0, 31.0, 24.0, 6.0, 36.0], 'gene_assignment': ['---', '---', 'NM_001005240 // OR4F17 // olfactory receptor, family 4, subfamily F, member 17 // 19p13.3 // 81099 /// NM_001004195 // OR4F4 // olfactory receptor, family 4, subfamily F, member 4 // 15q26.3 // 26682 /// NM_001005484 // OR4F5 //

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide which keys store identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'gene_assignment'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library
gene_data = apply_gene_mapping(gene_data, gene_mapping)
